# DQNの解説

![image.png](attachment:image.png)

画像引用:  
https://qiita.com/sugulu/items/3c7d6cbe600d455e853b

### DQNの特徴
- Q学習において状態行動テーブルを関数で表したもの.
- 離散的な行動を扱うことができる.

参考:  
http://blog.syundo.org/post/20171208-reinforcement-learning-dqn-and-impl/

### OpenAI gymのインストール

githubのレポジトリを参考に, gymモジュールをインストールしてください.  
https://github.com/openai/gym

In [1]:
import gym
import numpy as np
import renom as rm
import matplotlib.pyplot as plt
from renom.utility.initializer import Gaussian
from renom.cuda import set_cuda_active
from renom_rl.dqn import DQN
from renom_rl.env import BaseEnv
from gym.core import Env
from PIL import Image

set_cuda_active(True)
env = gym.make('BreakoutNoFrameskip-v4')

class CustomEnv(BaseEnv):
    
    def __init__(self, env):
        self.env = env
        self.action_shape = 4
        self.state_shape = (4, 84, 84)
        self.previous_frames = []
        super(CustomEnv, self).__init__()
    
    def reset(self):
        self.env.reset()
        n_step = np.random.randint(4, 32+1)
        for _ in range(n_step):
            state, _, _ = self.step(self.env.action_space.sample())
        return state
    
    def sample(self):
        return self.env.action_space.sample()
    
    def render(self):
        self.env.render()

    def _preprocess(self,state):
        resized_image = Image.fromarray(state).resize((84, 84)).convert('L')
        # image_array = np.asarray(resized_image.getdata()).reshape(110, 84)/255.
        # final_image = image_array[26:110, :]
        final_image = np.asarray(resized_image.getdata()).reshape(84, 84)/255.
        return final_image
    
    def step(self, action):
        state_list = []
        reward_list = []
        terminal = False
        for _ in range(4):
            # Use last frame. Other frames will be skipped.
            s, r, t, _ = self.env.step(action)
            state = self._preprocess(s)
            reward_list.append(r)
            if t:
                terminal = True
                
        if len(self.previous_frames) > 3:
            self.previous_frames = self.previous_frames[1:] + [state]
        else:
            self.previous_frames += [state]
        state = np.stack(self.previous_frames)
        return state, np.sum(reward_list), terminal
    
custom_env = CustomEnv(env)
q_network = rm.Sequential([rm.Conv2d(32, filter=8, stride=4, ignore_bias=True),
                           rm.Relu(),
                           rm.Conv2d(64, filter=4, stride=2, ignore_bias=True),
                           rm.Relu(),
                           rm.Conv2d(64, filter=3, stride=1, ignore_bias=True),
                           rm.Relu(), 
                           rm.Flatten(), 
                           rm.Dense(512, ignore_bias=True),
                           rm.Relu(),
                           rm.Dense(custom_env.action_shape, ignore_bias=True)])

In [2]:
model = DQN(custom_env, q_network)

In [3]:
model.fit(render=True, greedy_step=1000000)

Run random 5000 step for storing experiences


episode 001 avg_loss: 0.005 total_reward [train:1.000 test:-] e-greedy:0.000: : 161it [00:04, 43.29it/s]
episode 002 avg_loss: 0.010 total_reward [train:0.000 test:-] e-greedy:0.000: : 115it [00:02, 44.38it/s]
episode 003 avg_loss: 0.007 total_reward [train:2.000 test:-] e-greedy:0.000: : 208it [00:04, 42.52it/s]
episode 004 avg_loss: 0.006 total_reward [train:0.000 test:-] e-greedy:0.001: : 133it [00:03, 42.42it/s]
episode 005 avg_loss: 0.003 total_reward [train:3.000 test:-] e-greedy:0.001: : 216it [00:04, 43.73it/s]
episode 006 avg_loss: 0.005 total_reward [train:0.000 test:-] e-greedy:0.001: : 124it [00:02, 42.76it/s]
episode 007 avg_loss: 0.007 total_reward [train:1.000 test:-] e-greedy:0.001: : 154it [00:03, 42.29it/s]
episode 008 avg_loss: 0.005 total_reward [train:2.000 test:-] e-greedy:0.001: : 170it [00:03, 43.19it/s]
episode 009 avg_loss: 0.004 total_reward [train:1.000 test:-] e-greedy:0.001: : 143it [00:03, 43.61it/s]
episode 010 avg_loss: 0.004 total_reward [train:0.000 t

episode 155 avg_loss: 0.003 total_reward [train:0.000 test:-] e-greedy:0.022: : 104it [00:02, 41.09it/s]
episode 156 avg_loss: 0.002 total_reward [train:0.000 test:-] e-greedy:0.022: : 124it [00:02, 43.20it/s]
episode 157 avg_loss: 0.001 total_reward [train:2.000 test:-] e-greedy:0.022: : 219it [00:05, 43.13it/s]
episode 158 avg_loss: 0.002 total_reward [train:0.000 test:-] e-greedy:0.022: : 133it [00:03, 42.40it/s]
episode 159 avg_loss: 0.001 total_reward [train:0.000 test:-] e-greedy:0.022: : 118it [00:02, 41.52it/s]
episode 160 avg_loss: 0.003 total_reward [train:1.000 test:-] e-greedy:0.023: : 153it [00:03, 43.03it/s]
episode 161 avg_loss: 0.002 total_reward [train:3.000 test:-] e-greedy:0.023: : 241it [00:05, 43.03it/s]
episode 162 avg_loss: 0.002 total_reward [train:1.000 test:-] e-greedy:0.023: : 147it [00:03, 43.34it/s]
episode 163 avg_loss: 0.001 total_reward [train:0.000 test:-] e-greedy:0.023: : 121it [00:02, 44.01it/s]
episode 164 avg_loss: 0.002 total_reward [train:3.000 t

episode 233 avg_loss: 0.002 total_reward [train:3.000 test:-] e-greedy:0.033: : 237it [00:05, 43.95it/s]
episode 234 avg_loss: 0.003 total_reward [train:0.000 test:-] e-greedy:0.033: : 131it [00:03, 42.93it/s]
episode 235 avg_loss: 0.002 total_reward [train:3.000 test:-] e-greedy:0.033: : 232it [00:05, 43.76it/s]
episode 236 avg_loss: 0.001 total_reward [train:3.000 test:-] e-greedy:0.034: : 214it [00:04, 43.68it/s]
episode 237 avg_loss: 0.002 total_reward [train:2.000 test:-] e-greedy:0.034: : 180it [00:04, 44.22it/s]
episode 238 avg_loss: 0.003 total_reward [train:0.000 test:-] e-greedy:0.034: : 121it [00:02, 42.37it/s]
episode 239 avg_loss: 0.002 total_reward [train:1.000 test:-] e-greedy:0.034: : 130it [00:02, 44.51it/s]
episode 240 avg_loss: 0.001 total_reward [train:1.000 test:-] e-greedy:0.034: : 148it [00:03, 43.78it/s]
episode 241 avg_loss: 0.002 total_reward [train:2.000 test:-] e-greedy:0.034: : 226it [00:05, 43.99it/s]
episode 242 avg_loss: 0.001 total_reward [train:0.000 t

episode 388 avg_loss: 0.002 total_reward [train:1.000 test:-] e-greedy:0.055: : 168it [00:03, 43.80it/s]
episode 389 avg_loss: 0.001 total_reward [train:1.000 test:-] e-greedy:0.055: : 162it [00:03, 43.23it/s]
episode 390 avg_loss: 0.001 total_reward [train:0.000 test:-] e-greedy:0.055: : 125it [00:02, 44.59it/s]
episode 391 avg_loss: 0.003 total_reward [train:0.000 test:-] e-greedy:0.055: : 103it [00:02, 43.86it/s]
episode 392 avg_loss: 0.002 total_reward [train:2.000 test:-] e-greedy:0.055: : 177it [00:04, 43.36it/s]
episode 393 avg_loss: 0.001 total_reward [train:2.000 test:-] e-greedy:0.056: : 201it [00:04, 44.08it/s]
episode 394 avg_loss: 0.001 total_reward [train:0.000 test:-] e-greedy:0.056: : 132it [00:03, 43.66it/s]
episode 395 avg_loss: 0.002 total_reward [train:1.000 test:-] e-greedy:0.056: : 172it [00:03, 43.51it/s]
episode 396 avg_loss: 0.002 total_reward [train:1.000 test:-] e-greedy:0.056: : 161it [00:03, 43.62it/s]
episode 397 avg_loss: 0.002 total_reward [train:2.000 t

...

episode 14898 avg_loss: 0.007 total_reward [train:6.000 test:-] e-greedy:0.900: : 326it [00:16, 19.76it/s]
episode 14899 avg_loss: 0.007 total_reward [train:7.000 test:-] e-greedy:0.900: : 378it [00:16, 21.16it/s]
episode 14900 avg_loss: 0.007 total_reward [train:7.000 test:5.000] e-greedy:0.900: : 398it [00:20, 21.63it/s]
episode 14901 avg_loss: 0.007 total_reward [train:4.000 test:-] e-greedy:0.900: : 270it [00:11, 22.93it/s]
episode 14902 avg_loss: 0.007 total_reward [train:6.000 test:-] e-greedy:0.900: : 338it [00:14, 23.39it/s]
episode 14903 avg_loss: 0.007 total_reward [train:4.000 test:-] e-greedy:0.900: : 226it [00:09, 22.81it/s]
episode 14904 avg_loss: 0.007 total_reward [train:6.000 test:-] e-greedy:0.900: : 333it [00:14, 20.48it/s]
episode 14905 avg_loss: 0.007 total_reward [train:5.000 test:-] e-greedy:0.900: : 299it [00:14, 20.74it/s]
episode 14906 avg_loss: 0.007 total_reward [train:10.000 test:-] e-greedy:0.900: : 350it [00:18, 19.02it/s]
episode 14907 avg_loss: 0.007 to

episode 15050 avg_loss: 0.007 total_reward [train:5.000 test:5.000] e-greedy:0.900: : 275it [00:24, 21.64it/s]
episode 15051 avg_loss: 0.008 total_reward [train:3.000 test:-] e-greedy:0.900: : 202it [00:14, 13.74it/s]
episode 15052 avg_loss: 0.007 total_reward [train:3.000 test:-] e-greedy:0.900: : 232it [00:16, 19.69it/s]
episode 15053 avg_loss: 0.008 total_reward [train:7.000 test:-] e-greedy:0.900: : 333it [00:15, 21.19it/s]
episode 15054 avg_loss: 0.008 total_reward [train:6.000 test:-] e-greedy:0.900: : 307it [00:19, 15.35it/s]
episode 15055 avg_loss: 0.008 total_reward [train:5.000 test:-] e-greedy:0.900: : 313it [00:18, 17.35it/s]
episode 15056 avg_loss: 0.008 total_reward [train:10.000 test:-] e-greedy:0.900: : 491it [00:35, 13.72it/s]
episode 15057 avg_loss: 0.007 total_reward [train:8.000 test:-] e-greedy:0.900: : 408it [00:24, 16.66it/s]
episode 15058 avg_loss: 0.008 total_reward [train:7.000 test:-] e-greedy:0.900: : 364it [00:17, 20.64it/s]
episode 15059 avg_loss: 0.008 to

episode 15202 avg_loss: 0.007 total_reward [train:5.000 test:-] e-greedy:0.900: : 297it [00:17, 16.56it/s]
episode 15203 avg_loss: 0.007 total_reward [train:7.000 test:-] e-greedy:0.900: : 372it [00:18, 22.00it/s]
episode 15204 avg_loss: 0.007 total_reward [train:3.000 test:-] e-greedy:0.900: : 185it [00:08, 21.27it/s]
episode 15205 avg_loss: 0.007 total_reward [train:4.000 test:-] e-greedy:0.900: : 241it [00:16, 20.72it/s]
episode 15206 avg_loss: 0.008 total_reward [train:10.000 test:-] e-greedy:0.900: : 470it [00:27, 19.32it/s]
episode 15207 avg_loss: 0.007 total_reward [train:5.000 test:-] e-greedy:0.900: : 292it [00:13, 21.26it/s]
episode 15208 avg_loss: 0.007 total_reward [train:7.000 test:-] e-greedy:0.900: : 363it [00:22, 16.40it/s]
episode 15209 avg_loss: 0.007 total_reward [train:3.000 test:-] e-greedy:0.900: : 222it [00:10, 22.05it/s]
episode 15210 avg_loss: 0.007 total_reward [train:5.000 test:-] e-greedy:0.900: : 284it [00:23, 17.95it/s]
episode 15211 avg_loss: 0.007 total_

episode 15354 avg_loss: 0.007 total_reward [train:11.000 test:-] e-greedy:0.900: : 514it [00:31, 16.57it/s]
episode 15355 avg_loss: 0.007 total_reward [train:4.000 test:-] e-greedy:0.900: : 282it [00:13, 21.10it/s]
episode 15356 avg_loss: 0.007 total_reward [train:6.000 test:-] e-greedy:0.900: : 309it [00:18, 12.51it/s]
episode 15357 avg_loss: 0.007 total_reward [train:7.000 test:-] e-greedy:0.900: : 372it [00:20, 18.59it/s]
episode 15358 avg_loss: 0.008 total_reward [train:6.000 test:-] e-greedy:0.900: : 332it [00:15, 21.77it/s]
episode 15359 avg_loss: 0.008 total_reward [train:8.000 test:-] e-greedy:0.900: : 370it [00:17, 20.98it/s]
episode 15360 avg_loss: 0.007 total_reward [train:9.000 test:-] e-greedy:0.900: : 475it [00:25, 18.44it/s]
episode 15361 avg_loss: 0.007 total_reward [train:5.000 test:-] e-greedy:0.900: : 303it [00:17, 17.52it/s]
episode 15362 avg_loss: 0.007 total_reward [train:5.000 test:-] e-greedy:0.900: : 269it [00:12, 13.80it/s]
episode 15363 avg_loss: 0.008 total_

KeyboardInterrupt: 

In [4]:
q_network.save("dqn_exp5.h5")
# model = DQN(custom_env, q_network)

In [ ]:
model.test(render=True)

In [ ]:
import time
start_t = time.time()
a = np.random.permutation(int(1e1))
print(time.time()-start_t)

# 